In [8]:
import sys

# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/Users/mac/Desktop/PycharmProjects/TAADL/src')

In [14]:
from utils import train_test_split2, generate_portfolio_inputs
from network import MLP, PortfolioLayer
from config import DATA_PATH, PARA_PATH

import numpy as np
import pandas as pd
import torch
import torch.nn as nn

In [15]:
# import data and trained network parameters
features = pd.read_csv(DATA_PATH, index_col='Date')
net_params = torch.load(PARA_PATH)['net_state_dict']

In [16]:
tr_X, tr_y, te_X, te_y = train_test_split2(features)
pf_feature = generate_portfolio_inputs(tr_X, tr_y)

In [17]:
pf_feature.head()

,BNDX_exp_return,BND_exp_return,VGK_exp_return,VNQI_exp_return,VNQ_exp_return,VTI_exp_return,VWOB_exp_return,VWO_exp_return,BNDX_uncertainty,BND_uncertainty,...,VWOB_uncertainty,VWO_uncertainty,BNDX_mae,BND_mae,VGK_mae,VNQI_mae,VNQ_mae,VTI_mae,VWOB_mae,VWO_mae
Date,,,,,,,,,,,,,,,,,,,,,
2007-01-18,0.009079,0.011690,-0.014164,0.017123,0.021247,0.015664,0.007445,0.049585,0.004694,0.004369,...,0.010093,0.013021,0.013716,0.018295,0.142645,0.086824,0.177926,0.050222,0.007779,0.263713
2007-01-19,0.009602,0.011336,-0.018073,0.011836,0.016679,0.012137,0.004993,0.044295,0.004082,0.003883,...,0.009084,0.012877,0.013257,0.016077,0.119994,0.065336,0.187930,0.036726,0.003846,0.216212
2007-01-22,0.009370,0.011266,-0.017432,0.012273,0.017088,0.012912,0.005660,0.045205,0.004424,0.003997,...,0.008991,0.012992,0.013287,0.015644,0.105955,0.053302,0.198890,0.026307,0.004612,0.195689
2007-01-23,0.010180,0.011197,-0.022084,0.006175,0.011300,0.008790,0.003601,0.038257,0.003467,0.003259,...,0.008484,0.011775,0.013556,0.015677,0.102774,0.052490,0.199353,0.025795,0.003482,0.194642
2007-01-24,0.010350,0.010994,-0.025339,0.001537,0.007384,0.006194,0.001565,0.033862,0.003272,0.003159,...,0.008214,0.011182,0.013648,0.015135,0.101245,0.050761,0.201955,0.020578,0.002860,0.188355
